In [22]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
actors_df = pd.read_csv('campaign_1_actors.csv')


In [ ]:
def cleanNaNs(df):
    # change NaN to 'None'
    df = df.fillna('-')
    return df

def loc_window(df,t, idx, size):
    if t == 'iloc':
        return df.iloc[idx-size:idx+size+1]
    elif t == 'loc':
        return df.loc[idx-size:idx+size+1]
    return 'Invalid type'

In [ ]:
df_combat = pd.read_csv('df_combat_icooc_v2_manual.csv')

In [ ]:
df_combat = cleanNaNs(df_combat)

In [ ]:
df_combat[df_combat['item'] != 'None']

In [ ]:
loc_window(df_combat, 'iloc', 10, 10)

In [ ]:
actors_df = actors_df.drop_duplicates(subset=['actor_id'], keep='first')
actors_df.rename(columns={'actor_id': 'speaker'}, inplace=True)


In [ ]:
actors_df

In [ ]:

# merge without dropping NaNs and do not sort

df_combined = pd.merge(df_combat, actors_df, how='left', on='speaker', sort=False)



In [ ]:
df_combined = df_combined.drop(columns=['actor'])

In [ ]:
# reorder columns of df_combined

df_combined = df_combined[['fileID', 'number', 'speaker', 'character', 'Race', 'class', 'utterance', 'item', 'owner', 'in_combat', 'IC_OOC']]



In [ ]:
df_combined = cleanNaNs(df_combined)

In [23]:
# df_combined.loc[0:10]

# display column utterance wider
pd.set_option('display.max_colwidth', None)

In [ ]:
combine_w_item = df_combined[(df_combined['item'] != 'None') & (df_combined['item'] != 'Whisper')]

In [ ]:
combine_w_item


In [ ]:
loc_window(combine_w_item, 'iloc', 900, 10)

In [ ]:
df_combined

In [ ]:
df_combined = cleanNaNs(df_combined)

In [ ]:
# df_combined.to_csv('extracted_v1.csv', index=False)

In [24]:
df_ = pd.read_csv('extracted_v1.csv')

In [31]:
df_[df_['owner'] != '-'].loc[60380:60381]

,fileID,number,speaker,character,Race,class,utterance,item,owner,in_combat,IC_OOC
60380,C1E026,215,<PLAYER5>,Tiberius Stormwind,Dragonborn,Sorcerer (Draconic Bloodline),"Well, you've seen how I've used my Mending Wheel. Countless ways.",Mending Wheel,<PLAYER5>,True,OOC


fileID                                                  C2E041
number                                                     100
speaker                                              <PLAYER6>
character                                            Vax'ildan
Race                                                  Half-elf
class        Rogue (Assassin)/Paladin (Oath of Vengeance)/D...
utterance    Periapt of wound closure. It just feels right ...
item                                  Periapt of Wound Closure
owner                                                <PLAYER6>
in_combat                                                 True
IC_OOC                                                     OOC
Name: 382809, dtype: object

In [ ]:
df_.drop(columns=['in_combat'], inplace=True)

In [ ]:
loc_window(df_, 'iloc', 10, 10)

In [ ]:
# output format

# message = [
#     {"role": "<PLAYER1>", "content": "the goblin is attacking me! can i run away?"},
#     {"role": "<DM>", "content": "no, you have a slowness debuff, you can't run away."},
#     {"role": "<PLAYER2>", "content": "i will cast fireball on the goblin."},
# ]

In [ ]:
# c1_df = df_[df_['fileID'][:2] == 'C1']
# fileID starts with C1
df_ = df_[df_['fileID'].str.startswith('C1')]


In [ ]:
df_

# MASK NAMES


In [ ]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

def mask_with_ner(text):
    doc = nlp(text)
    new_text = text
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            new_text = new_text.replace(ent.text, '<CHARACTER>')
    return new_text

def mask_character_names(row):
    # masked_data = []
    masked_row = mask_with_ner(row)
        # Also replace character metadata manually
        # content = re.sub(r'\[character:.*?\]', '[character: <CHARACTER>]', content)
    return masked_row

# Example
# data = [{'role': 'user', 'content': '"Hi I am Vex!" [character: Vex\'ahlia, in_context: False]'}]



In [ ]:
# df_mask_test = df_.iloc[10:30].copy()
from tqdm import tqdm

In [ ]:
# tqdm.pandas(desc="Masking character names...")
# df_['utterance'] = df_['utterance'].progress_apply(mask_character_names)

In [ ]:
df_.to_csv('extracted_v2_char_masked.csv', index=False)

In [ ]:
# win_size = 8

# # tqdm step size is win_size
# to_json = []
# for i in tqdm(range(40, df_.shape[0] , win_size), desc="Processing"):

#     # get the current window
#     df_window = df_.iloc[i:i+win_size]

#     win_data = []
#     for idx, row in df_window.iterrows():
#         # index 
#         original_index = row['number']
#         speaker = row['speaker']
#         character = row['character']
#         class_ = row['class']
#         race = row['Race']
#         item = row['item']
#         # owner = row['owner']
#         utterance = row['utterance']
#         in_context = "True" if row['IC_OOC'] == 'IC' else "False"

#         # "I cast Faerie Fire on the goblin." [race: Half-elf, class: Druid, IC]

#         # content = f"\"{utterance}\" [character: {character}, class: {class_}, race: {race}, in_context: {in_context}, items: {item}]"
#         content = f"{utterance}"
#         role = "assistant" if speaker == '<DM>' else "user"

#         message = {
#             # "original_index": original_index,
#             "role": role,
#             "content": content
#         }

#         win_data.append(message)
    
#     # add the window data to the to_json list
#     to_json.append(win_data)
    
# import json
# with open('extracted_to_llama_v1.json', 'w') as f:
#     json.dump(to_json, f, indent=4)


In [ ]:
# to_json[:4]

In [ ]:
# import json
# with open('extracted_to_llama_v3.json', 'w') as f:
#     json.dump(to_json, f, indent=4)

In [ ]:
df

In [ ]:
def window_data_with_single_dm(df):
    to_json = []
    cur_idx = 0
    win_data = []
    while cur_idx < df.shape[0]:
        row = df.iloc[cur_idx]
        utterance = row['utterance']
        content = f"{utterance}"
        role = "assistant" if row['speaker'] == '<DM>' else row['speaker']
        message = {
                "role": role,
                "content": content
            }
        
        win_data.append(message)
        cur_idx += 1
        if row['speaker'] == '<DM>':
            if row['IC_OOC'] == 'IC':
                if len(win_data) > 15:
                    # print(len(win_data))
                    win_data = []
                    continue
                to_json.append(win_data)
                win_data = []
            else:
                # ooc
                win_data = []
    return to_json

            

In [ ]:
df = pd.read_csv('extracted_v2_char_masked.csv')

In [ ]:
to_json = window_data_with_single_dm(df)

In [ ]:
to_save = to_json[2:]

In [ ]:

to_save[:3]

In [ ]:
maxlen = 0
for i , c in enumerate(to_save):
    # findmaximum length of the element in the list of dictionaries
    # print(len(c), i)
    if len(c) > maxlen:
        maxlen = len(c)
        print(len(c), i) 
    

In [ ]:
# to_save[30238]

In [ ]:
len(to_save)

In [ ]:
# 34555

In [ ]:
import json

with open('extracted_to_llama_v5_IC.json', 'w') as f:
    json.dump(to_save, f, indent=4)

In [ ]:
# import json

# # Load from file
# with open("extracted_to_llama_v1.json", "r", encoding="utf-8") as f:
#     messages = json.load(f)

# print(messages[0]['role'])